In [1]:
import pandas as pd
import numpy as np

In [52]:
Parkhäuser = pd.read_csv("../Final_Data/Cleaned/1000014cleaned.csv")
Flugbewegungen = pd.read_csv("../Final_Data/Cleaned/Airport_traffic_hourly_cleaned_std.csv")
#noch Probleme mit dem Datensatz: Gasverbrauch = pd.read_csv("../Final_Data/Cleaned/Gasverbrauch_cleaned.csv")
Schallmessungen = pd.read_csv("../Final_Data/Cleaned/Schallmessungen_cleaned.csv", sep=";")
Stromverbrauch = pd.read_csv("../Final_Data/Cleaned/Stromverbrauch_Basel_hourly_cleaned.csv")
Wetterdaten = pd.read_csv("../Final_Data/Cleaned/Wetterdaten_Basel_cleaned22_24.csv")
Verkehrsdaten = pd.read_csv("../Final_Data/Cleaned/merged_df.csv")

In [53]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
Verkehrsdaten.head()

,Datum,350nPW,350nPW+,350nLief,350nLief+,350nLief+Aufl.,350nLW,350nLW+,350nSattelzug,350nBus,...,84111108vPW,84111108vPW+,84111108vLief,84111108vLief+,84111108vLief+Aufl.,84111108vLW,84111108vLW+,84111108vSattelzug,84111108vBus,84111108vandere
0,2021-12-31 23:00:00+00:00,241.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2022-01-01 00:00:00+00:00,218.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,...,64.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-01-01 01:00:00+00:00,157.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-01-01 02:00:00+00:00,109.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-01-01 03:00:00+00:00,96.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
Parkhäuser.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
0,2022-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-01-01 01:00:00+00:00,7.0,6.0,15.0,19.0,0.0,2.0,2.0,3.0,1.0,23.0,5.0,10.0,4.0,7.0,0.0,2.0
2,2022-01-01 02:00:00+00:00,7.0,1.0,4.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,9.0,19.0,0.0,8.0,1.0,1.0
3,2022-01-01 03:00:00+00:00,4.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,12.0,23.0,0.0,8.0,0.0,0.0
4,2022-01-01 04:00:00+00:00,3.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,10.0,0.0,2.0,0.0,0.0


In [55]:
Flugbewegungen.head()

,Datum,Hour,Traffic,Weekday
0,2022-01-01 00:00:00+00:00,0,-1.142151,Saturday
1,2022-01-01 00:00:00+00:00,1,-1.295479,Saturday
2,2022-01-01 00:00:00+00:00,2,-1.295479,Saturday
3,2022-01-01 00:00:00+00:00,3,-1.295479,Saturday
4,2022-01-01 00:00:00+00:00,4,-1.295479,Saturday


In [56]:
Schallmessungen.head()

,Datum,Wert
0,2022-01-01 00:00:00+00:00,1.144333
1,2022-01-01 01:00:00+00:00,1.078958
2,2022-01-01 02:00:00+00:00,1.041345
3,2022-01-01 03:00:00+00:00,1.016109
4,2022-01-01 04:00:00+00:00,1.035645


In [ ]:
# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner')
#                    .merge(Flugbewegungen, on='Datum', how='inner') \
#                      .merge(Schallmessungen, on='Datum', how='inner') \ -> too many NaN values
#                      .merge(Gasverbrauch, on='Datum', how='inner') \ -> not working yet
#                      .merge(Stromverbrauch, on='Datum', how='inner') \
#                      .merge(Verkehrsdaten, on='Datum', how='inner') \
#                      .merge(Wetterdaten, on='Datum', how='inner')

# Display the first few rows of the merged dataframe
final_df.head(25)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
0,2022-01-01 00:00:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.612920,-0.179497,-0.470399,-0.507739
1,2022-01-01 01:00:00+00:00,7.0,6.0,15.0,19.0,0.0,2.0,2.0,3.0,1.0,...,5.0,10.0,4.0,7.0,0.0,2.0,-0.632843,-0.179497,-0.400364,-0.580393
2,2022-01-01 02:00:00+00:00,7.0,1.0,4.0,9.0,1.0,0.0,0.0,0.0,0.0,...,9.0,19.0,0.0,8.0,1.0,1.0,-0.652767,-0.179497,-0.418171,-0.696911
3,2022-01-01 03:00:00+00:00,4.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,12.0,23.0,0.0,8.0,0.0,0.0,-0.708802,-0.179497,-0.473559,-0.780239
4,2022-01-01 04:00:00+00:00,3.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,...,15.0,10.0,0.0,2.0,0.0,0.0,-0.747404,-0.179497,-0.470399,-0.886083
5,2022-01-01 05:00:00+00:00,2.0,0.0,-2.0,-2.0,0.0,0.0,-2.0,0.0,-1.0,...,1.0,7.0,0.0,2.0,0.0,-1.0,-0.818382,-0.179497,-0.425181,-0.961834
6,2022-01-01 06:00:00+00:00,-2.0,0.0,-85.0,4.0,0.0,-1.0,0.0,-1.0,0.0,...,5.0,2.0,0.0,-4.0,1.0,0.0,-0.876908,-0.179497,-0.382857,-0.971394
7,2022-01-01 07:00:00+00:00,1.0,-1.0,2.0,2.0,0.0,-1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,-4.0,0.0,-3.0,-0.946641,-0.179497,-0.324656,-0.956170
8,2022-01-01 08:00:00+00:00,0.0,1.0,-5.0,1.0,0.0,0.0,-1.0,-1.0,1.0,...,-3.0,-3.0,0.0,-4.0,0.0,0.0,-1.013883,-0.179497,-0.311026,-1.012241
9,2022-01-01 09:00:00+00:00,0.0,-1.0,-13.0,0.0,0.0,0.0,-5.0,1.0,1.0,...,-8.0,2.0,0.0,-2.0,0.0,-5.0,-0.985243,-0.179497,-0.254630,-0.995954


In [67]:
# Check for rows with NaN values
nan_rows = final_df[final_df.isnull().any(axis=1)]

# Display the rows with NaN values
nan_rows.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
17280,2023-12-22 00:00:00+00:00,1.0,0.0,10.0,5.0,0.0,0.0,2.0,1.0,-1.0,...,19.0,26.0,4.0,2.0,2.0,5.0,NaN,NaN,NaN,NaN
17281,2023-12-22 01:00:00+00:00,0.0,0.0,3.0,1.0,0.0,0.0,2.0,0.0,1.0,...,1.0,11.0,0.0,1.0,0.0,12.0,NaN,NaN,NaN,NaN
17282,2023-12-22 02:00:00+00:00,0.0,0.0,2.0,0.0,0.0,0.0,0.0,-2.0,0.0,...,5.0,9.0,1.0,0.0,0.0,-3.0,NaN,NaN,NaN,NaN
17283,2023-12-22 03:00:00+00:00,0.0,0.0,0.0,5.0,0.0,0.0,0.0,2.0,0.0,...,7.0,5.0,0.0,0.0,0.0,-12.0,NaN,NaN,NaN,NaN
17284,2023-12-22 04:00:00+00:00,0.0,0.0,-4.0,0.0,0.0,0.0,-1.0,-2.0,1.0,...,-2.0,6.0,0.0,0.0,0.0,-41.0,NaN,NaN,NaN,NaN


In [31]:
final_df.to_csv(r"C:\Users\maxd2\OneDrive - Universitaet St.Gallen\Desktop\DSF\DataFrame_Basel_Project.csv", index=False)

In [ ]:
#2 NaN in Stromverbrauch 2022-10-30 01:00:00 & 2023-10-29 01:00:00

# Verkehrsdatenset hat 5 NaN values 2024-10-20 22:00:00 & 2024-10-20 23:00:00 & 2024-10-21 00:00:00 & 2024-10-21 01:00:00 & 2024-10-21 02:00:00	

# Wetter hat 5 NaN values 2023-12-22 00:00:00 & 2023-12-22 01:00:00 & 2023-12-22 02:00:00 & 2023-12-22 03:00:00 & 2023-12-22 04:00:00
# habe nochmal nachgeschaut und es ist der ganze 22 und 23 Dezember 2023 || ist aber nur eine Auflösung die ich ausgewählt habe, sollte mit der anderen Auflösung nicht mehr vorkommen
